# CDAT Migration Regression Testing Notebook

Comparing `cdat-migration-fy24` against `main` without the slice_flag.

FINDNGS: Results are all nearly identical.


## Setup Code


In [16]:
import glob
from auxiliary_tools.cdat_regression_testing.utils import (
    get_metrics,
    get_rel_diffs,
    get_num_metrics_above_diff_thres,
    highlight_large_diffs,
    sort_columns,
    update_diffs_to_pct,
    PERCENTAGE_COLUMNS,
)

import pandas as pd

# TODO: Update DEV_RESULTS and MAIN_RESULTS to your diagnostic sets.
DEV_PATH = "/global/cfs/cdirs/e3sm/www/vo13/examples_658/ex1_modTS_vs_modTS_3years/lat_lon/model_vs_model"
MAIN_PATH = "/global/cfs/cdirs/e3sm/www/vo13/examples_main_no_slice/ex1_modTS_vs_modTS_3years/lat_lon/model_vs_model"

DEV_GLOB = sorted(glob.glob(DEV_PATH + "/*.json"))
MAIN_GLOB = sorted(glob.glob(MAIN_PATH + "/*.json"))

## 1. Get the metrics for the development and `main` branches and their differences.


In [50]:
df_metrics_dev = get_metrics(DEV_GLOB)
df_metrics_main = get_metrics(MAIN_GLOB)

df_metrics_dev2 = df_metrics_dev.reset_index(names=["var_key", "metric"])
df_metrics_dev2 = df_metrics_dev2.loc[
    df_metrics_dev2.var_key.isin(df_metrics_main.index.get_level_values(0).unique())
]
df_metrics_dev2 = df_metrics_dev2.set_index(["var_key", "metric"])


df_metrics_diffs = get_rel_diffs(df_metrics_dev2, df_metrics_main)

In [51]:
df_metrics_diffs

test DIFF (%)  ref DIFF (%)  test_regrid DIFF (%)  \
var_key      metric                                                      
LHFLX        min      0.000000e+00  0.000000e+00          0.000000e+00   
             max      0.000000e+00  2.061090e-16          0.000000e+00   
             mean     1.607934e-15  1.118090e-15          1.607934e-15   
             std               NaN           NaN          5.414602e-16   
             rmse              NaN           NaN                   NaN   
             corr              NaN           NaN                   NaN   
LWCF         min      0.000000e+00  0.000000e+00          0.000000e+00   
             max      0.000000e+00  3.300170e-16          0.000000e+00   
             mean     1.894919e-15  1.310068e-15          1.894919e-15   
             std               NaN           NaN          6.638807e-16   
             rmse              NaN           NaN                   NaN   
             corr              NaN           NaN                   NaN   
NET_FLUX_SRF min      0.000000e+00  0.000000e+00          0.000000e+00   
             max      1.825516e-16  1.706434e-16          1.825516e-16   
             mean     2.535940e-15  1.465413e-14          2.535940e-15   
             std               NaN           NaN          3.531975e-16   
             rmse              NaN           NaN                   NaN   
             corr              NaN           NaN                   NaN   
PRECT        min      0.000000e+00  0.000000e+00          0.000000e+00   
             max      2.054785e-16  3.506280e-16          2.054785e-16   
             mean     1.308796e-15  1.010973e-15          1.308796e-15   
             std               NaN           NaN          4.107430e-16   
             rmse              NaN           NaN                   NaN   
             corr              NaN           NaN                   NaN   
PSL          min      0.000000e+00  1.168177e-16          0.000000e+00   
             max      2.216200e-16  0.000000e+00          2.216200e-16   
             mean     1.124213e-15  1.236728e-15          1.124213e-15   
             std               NaN           NaN          5.030916e-16   
             rmse              NaN           NaN                   NaN   
             corr              NaN           NaN                   NaN   
RESTOM       min      2.226235e-16  0.000000e+00          2.226235e-16   
             max      0.000000e+00  1.620247e-16          0.000000e+00   
             mean     6.224919e-15  1.107688e-13          6.224919e-15   
             std               NaN           NaN          3.940225e-16   
             rmse              NaN           NaN                   NaN   
             corr              NaN           NaN                   NaN   
TREFHT       min      2.525625e-16  1.221719e-16          2.525625e-16   
             max      1.140829e-16  1.191418e-16          1.140829e-16   
             mean     1.443220e-15  1.026647e-15          1.443220e-15   
             std               NaN           NaN          3.950659e-16   
             rmse              NaN           NaN                   NaN   
             corr              NaN           NaN                   NaN   
             min      2.525625e-16  1.221719e-16          2.525625e-16   
             max      1.140829e-16  1.191418e-16          1.140829e-16   
             mean     9.253642e-15  8.350693e-15          9.253642e-15   
             std               NaN           NaN          4.922302e-15   
             rmse              NaN           NaN                   NaN   
             corr              NaN           NaN                   NaN   

                     ref_regrid DIFF (%)  diff DIFF (%)  misc DIFF (%)  
var_key      metric                                                     
LHFLX        min            0.000000e+00   2.066978e-16            NaN  
             max            2.061090e-16   0.000000e+00            NaN  
             mean           

## 2. Filter differences to those above maximum threshold (2%).

All values below maximum threshold will be labeled as `NaN`.

- **If all cells in a row are NaN (< 2%)**, the entire row is dropped to make the results easier to parse.
- Any remaining NaN cells are below < 2% difference and **should be ignored**.


In [56]:
df_metrics_diffs_thres = df_metrics_diffs[df_metrics_diffs >= 0.02]
df_metrics_diffs_thres = df_metrics_diffs_thres.dropna(
    axis=0, how="all", ignore_index=False
)

In [57]:
df_metrics_diffs_thres

,,test DIFF (%),ref DIFF (%),test_regrid DIFF (%),ref_regrid DIFF (%),diff DIFF (%),misc DIFF (%)
var_key,metric,,,,,,


Findings: No metrics are above the 2% threshold after removing the `slice_flag` used in
the CDAT version of the codebase.
